<h1>CS786 Assignment-3 on Memory Models by Anmol Pabla (190154)<h1>
(Converted to Python)

# Q1: Filling in the partially complete code to make a TCM

The Temporal Context Model (TCM) takes into consideration the effect of the environment. It assumes a linear drift of the temporal context cue. The temporal context model assumes that the past becomes increasingly dissimilar to the future, so that memories become harder to retrieve the farther away in the past they are.

---

Here, we are going to model the world as a set of N continuous-valued features. We will model observations of states of the world as samples from N Gaussians with time-varying means and fixed variance. For simplicity, assume that agents change nothing in the world.

In [ ]:
# importing necessary files
import numpy as np
import random

In [ ]:
# Function provided in Matlab converted to python
def drawFromADist(p):
  if(np.sum(p)==0):
    p = 0.05*np.ones((1,len(p)))  
  p = p/np.sum(p)
  c = np.cumsum(p)
  # Choosing the minimum index that satisfies below condition
  idx = np.where((random.random() - np.cumsum(p))<0)
  sample = np.min(idx)
  out = np.zeros(p.size)
  # Index represented by sample is the value retrieved in this case
  out[sample] = 1
  return out

In [ ]:
def TCM(N_WORLD_FEATURES, N_ITEMS, ENCODING_TIME, TEST_TIME):

  # first fix the presentation schedule; Here its random 
  schedule = np.transpose([np.sort(np.round_(np.random.random(N_ITEMS)*ENCODING_TIME)), np.arange(0,N_ITEMS)])
  schedule_load = ENCODING_TIME/np.median(np.diff(schedule[:,1]))    

  # variable that acts as the proxy learning load on the agent
  encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES))

  # initial means
  world_m = [1,2,1,2,1]

  # initial variance
  world_var = 1
  delta = 0.05                       # causes linear drift in means
  beta_param = 0.001                 # used in the next part of the assignmnet
  m = 0


  for time in range(ENCODING_TIME):
      world_m = [k + delta for k in world_m]
      world = random.gauss(world_m, np.sqrt(world_var))
      #  items encoded in memory, in association with the
      #  state of the world at that time.
      if m<N_ITEMS:
          if(time==schedule[m,1]):
              encoding[m] = world                                                  
              # encode into the encoding vector
              m =  m + 1

  # simulating retrieval using SAM, but with a bijective image-item mapping

  out = np.empty(TEST_TIME)
  soa = np.empty(N_ITEMS)
  time = ENCODING_TIME

  while (time < ENCODING_TIME+TEST_TIME):
    # the state of the world is the retrieval cue
    world_m = [k + delta for k in world_m]
    world = random.gauss(world_m, np.sqrt(world_var))

    # model world evolution
    for m in range(0,N_ITEMS):
        soa[m] = np.dot(world,encoding[m])                                                                
        # finding association strengths

    out[time-ENCODING_TIME] = np.where(drawFromADist(soa))[0]
    time = time + 1

  success = (np.unique(out)).size                                     
  # success is number of unique retrievals
  return success

  #  humans can retrieve about 7 items effectively from memory

In [ ]:
avg_success = 0

# averaging success over 10 cycles
for i in range(10):
  curr_success = TCM(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20)
  avg_success += curr_success
  
avg_success = avg_success/10

print('Average Sucess: ',avg_success)

Average Sucess:  7.8


We find that the model manages to maintain an average success rate which is above 7, the value that is generally expected in humans. Thus, it can be said that the model works reasonably well.

---



# Q2: Sampling the rate of drift from a mixture of two Gaussians

One of the Gaussians has a smaller mean (about 0.05) and the other has an mean of about 2. Both the Gaussians have a variance of 1.

The values are chosen from this Gaussian mixture based on the beta parameter, values from the smaller meaned Gaussian are chosen with a probability of beta_param.

In [ ]:

def TCM_2(N_WORLD_FEATURES, N_ITEMS, ENCODING_TIME, TEST_TIME, TIMES):

  # first fix the presentation schedule passed as the argument TIMES in this case

  schedule = np.transpose([TIMES, np.arange(0,N_ITEMS)])
  schedule_load = ENCODING_TIME/np.median(np.diff(schedule[:,0]))          

  # variable proxy for learning load
  encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES))

  # Gaussian Means
  world_m = [1,2,1,2,1]

  world_var = 1
  # Averages: One bigger and one smaller
  delta_avg_large = 2                      
  delta_avg_small = 0.05
  # Parameter that defines probability of choosing smaller average from the bimodal mixture
  beta_param = 0.001                
  
  m = 0
  for time in range(ENCODING_TIME):
    d1 = random.gauss(delta_avg_large, np.sqrt(world_var))
    d2 = random.gauss(delta_avg_small, np.sqrt(world_var))
    # Choosing value from one of the Gaussian distributions based on beta_param
    delta = random.choices([d1,d2],weights=[beta_param,1-beta_param]) 

    world_m = [k + delta[0] for k in world_m]
    world = random.gauss(world_m, np.sqrt(world_var))

    if m<N_ITEMS:
        if(time==schedule[m,1]):
            encoding[m] = world                                                  
            # encode into the encoding vector
            m =  m + 1

  # simulating retrieval using SAM, but with a bijective image-item mapping
  out = np.empty(TEST_TIME)
  soa = np.empty(N_ITEMS)

  time = ENCODING_TIME
  while (time < ENCODING_TIME+TEST_TIME):
    # the state of the world is the retrieval cue
    # Modelling the drift from the Gaussians (Parameters known in this case)
    d1 = random.gauss(delta_avg_large, np.sqrt(world_var))
    d2 = random.gauss(delta_avg_small, np.sqrt(world_var))
    delta = random.choices([d1,d2],weights=[beta_param,1-beta_param]) 

    world_m = [k + delta[0] for k in world_m]
    world =  random.gauss(world_m, np.sqrt(world_var))

    # model world evolution
    for m in range(0,N_ITEMS):
        soa[m] = np.dot(world,encoding[m])                                                                
        # finding association strengths

    out[time-ENCODING_TIME] = np.where(drawFromADist(soa))[0]
    time = time + 1

  success = (np.unique(out)).size                                     
  # success is number of unique retrievals
  return success, schedule_load

  #  humans can retrieve about 7 items effectively from memory. 



---
Now we look at the retrieval rates for different Encoding Schedules:


First, we fix the presentaion schedule at the end of the encoding time.

In [ ]:
# Presentaion schedule shoved to the end
END_TIME = [491,492,493,494,495,496,497,498,499,500]

avg_success = 0
avg_load = 0
# averaging over 10 values
for i in range(10):
  curr_success, curr_load = TCM_2(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20, TIMES=END_TIME)
  avg_success += curr_success
  avg_load += curr_load

avg_success = avg_success/10
avg_load = avg_load/10

print('Average Sucess: ',avg_success)
print('Average Load: ',avg_load)

Average Sucess:  7.6
Average Load:  500.0


We see that the above case has a good Success rate but significantly high learning load (Maximum value of the load is obtained in the above case).



---
The minimum load occurs when the median value is as high as possible, this is seen when 5 items are scheduled at the end and the others are spaced with the biggest margin possible (about 100 units of time in this case)


In [ ]:
MIN_LOAD_TIME = [0,99,198,297,396,496,497,498,499,500]
avg_success = 0
avg_load = 0

for i in range(10):
  curr_success, curr_load = TCM_2(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20, TIMES=MIN_LOAD_TIME)
  avg_success += curr_success
  avg_load += curr_load

avg_success = avg_success/10
avg_load = avg_load/10

print('Average Sucess: ',avg_success)
print('Average Load: ',avg_load)

Average Sucess:  7.4
Average Load:  5.050505050505051


We find that the load is reduced to a minimum and the success rate is also decent. However, it is quite clear that we are exploiting the way the load is calculated and half of the presentations are still fixed at the end, and in real human scenarios this method would not lead to the minimum possible learning load. 

Acheiving a realisitic solution would possibly require modelling all events with an equal amount of time between them. This is what one expects to observe in the real world situation as well.


---



In [ ]:
MID_TIME = []
step = 500/10
time = 50

# times are equally spaced, the last value being at 500
while time<=500:
  MID_TIME.append(int(time))
  time = time + step
  
avg_success = 0
avg_load = 0 

# averaging over 10 values
for i in range(10):
  curr_success, curr_load = TCM_2(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20, TIMES=MID_TIME)
  avg_success += curr_success
  avg_load += curr_load

avg_success = avg_success/10
avg_load = avg_load/10

print('Average Sucess: ',avg_success)
print('Average Load: ',avg_load)

Average Sucess:  7.2
Average Load:  10.0


We seem to have achieved a reasonable position in our TCM with the load not being too high but managing to maintain a retrieval rate of about 7 items which is generally seen in humans.


---



# Q3: Modelling a similar case as Q2, but the retrieving agent doesn't know parameters of the Bimodal Distribution used to sample the drift.

---
Here we need to use the EM algorithm to learn the parameters of the bimodal distribution and allow the retrieval agent to use them. An expectation–maximization (EM) algorithm is an iterative method to find (local) maximum likelihood estimates of parameters in statistical models, where the model depends on unobserved latent variables. 

Here, we use the EM algorithm to learn the parameters of the mixed Gaussian distribution. The algorithm is an iterative algorithm that starts from some initial estimates of the parameters, and then proceeds to
iteratively update the values until convergence is detected. 




---
In this model, the EM algorithm is implemented using the GaussianMixture function from the Scikit-Learn Library. 

The function takes a distribution as an argument and infers Gaussians present in that distribution. The function is designed to detect n_components number of Gaussians which is also passed as an argument.

In [ ]:
from sklearn.mixture import GaussianMixture

# Function to learn the Gaussians in the Bimodal Distribution using the EM algorithm
# Return a lst of samples generated from the Gaussian mixture

def get_samples_from_EM(deltas):
  deltas = np.array(deltas)
  deltas = deltas.reshape(-1, 1)
  # Model learns using the distribution given as input
  gm = GaussianMixture(n_components=2, random_state=0).fit(deltas)
  # sampling 100 values using the learned parameters
  delta_samples = gm.sample(1000)
  delta_samples = delta_samples[0].reshape(-1)
  return delta_samples  

In [ ]:
def TCM_3(N_WORLD_FEATURES, N_ITEMS, ENCODING_TIME, TEST_TIME, TIMES):

  # first fix the presentation schedule passed using TIMES

  schedule = np.transpose([TIMES, np.arange(0,N_ITEMS)])
  schedule_load = ENCODING_TIME/np.median(np.diff(schedule[:,0]))                 
  # variable proxy for learning load
  encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES))

  world_m = [1,2,1,2,1]

  world_var = 1
  # small and large mean for Gaussians
  delta_avg_large = 2                      
  delta_avg_small = 0.05
  # beta_param to choose from the distrbution
  beta_param = 0.001                 
  m = 0

  # list to save the values of drift obtained using Ecnoding
  delta_data = []

  for time in range(ENCODING_TIME):
    d1 = random.gauss(delta_avg_large, np.sqrt(world_var))
    d2 = random.gauss(delta_avg_small, np.sqrt(world_var))
    # Choosing value from one of the Gaussian distributions based on beta_param
    delta = random.choices([d1,d2],weights=[beta_param,1-beta_param]) 
    delta_data.append(delta)

    world_m = [k + delta[0] for k in world_m]
    world = random.gauss(world_m, np.sqrt(world_var))
    #  Items encoed in association with the state of the world at that time.
    if m<N_ITEMS:
        if(time==schedule[m,1]):
            encoding[m] = world                                                  
            # encode into the encoding vector
            m =  m + 1

  # simulating retrieval using SAM, but with a bijective image-item mapping
  out = np.empty(TEST_TIME)
  soa = np.empty(N_ITEMS)

  # Learning the parameters of the Gaussian mixture and
  # getting list of values sampled from the distribution, 
  # these will be used for obtaining the drift 
  delta_samples = get_samples_from_EM(delta_data)

  time = ENCODING_TIME
  while (time < ENCODING_TIME+TEST_TIME):
    # the state of the world is the retrieval cue
    # randomly choosing the value of drift from the samples obtained above
    delta = random.choice(delta_samples) 

    world_m = [k + delta for k in world_m]
    world =  random.gauss(world_m, np.sqrt(world_var))
    # model world evolution
    for m in range(0,N_ITEMS):
        soa[m] = np.dot(world,encoding[m])                                                                
        # finding association strengths
    out[time-ENCODING_TIME] = np.where(drawFromADist(soa))[0]
    time = time + 1

  success = (np.unique(out)).size                                     
  # success is number of unique retrievals
  return success, schedule_load

  #  humans can retrieve about 7 items effectively from memory. get this model
  #  to behave like humans

Like in the previous Question, we use three Encoding schedules:


*   Shoving Presentations to the end of the encoding schedule, this leads to a high retrieval rate but also a significantly high load.
*   Maximizing the median gap in presentation schedule to minimize the Load.
*   Distibuting Presentations uniformly to try and get an optimal but realsitc Load and retrieval.



In [ ]:
# All presentations at the end of the schedule
END_TIME = [491,492,493,494,495,496,497,498,499,500]

avg_success = 0
avg_load = 0
for i in range(10):
  curr_success, curr_load = TCM_3(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20, TIMES=END_TIME)
  avg_success += curr_success
  avg_load += curr_load

avg_success = avg_success/10
avg_load = avg_load/10

print('Average Sucess: ',avg_success)
print('Average Load: ',avg_load)

Average Sucess:  7.5
Average Load:  500.0


We find a high load and high retrieval.

---



In [ ]:
# Getting the minimum load time by maximizing the median time difference
MIN_LOAD_TIME = [0,99,198,297,396,496,497,498,499,500]
avg_success = 0
avg_load = 0

for i in range(10):
  curr_success, curr_load = TCM_3(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20, TIMES=MIN_LOAD_TIME)
  avg_success += curr_success
  avg_load += curr_load

avg_success = avg_success/10
avg_load = avg_load/10

print('Average Sucess: ',avg_success)
print('Average Load: ',avg_load)

Average Sucess:  7.3
Average Load:  5.050505050505051


In the above case, the load is low but presentations are still towards the end. Next we move onto a realistic solution to the problem.

In [ ]:
# Realistic case: Equally spaced presentation
MID_TIME = []
step = 500/10
time = 50

while time<=500:
  MID_TIME.append(int(time))
  time = time + step
  
avg_success = 0
avg_load = 0 

for i in range(10):
  curr_success, curr_load = TCM_3(N_WORLD_FEATURES = 5, N_ITEMS = 10, ENCODING_TIME = 500, TEST_TIME = 20, TIMES=MID_TIME)
  avg_success += curr_success
  avg_load += curr_load

avg_success = avg_success/10
avg_load = avg_load/10

print('Average Sucess: ',avg_success)
print('Average Load: ',avg_load)

Average Sucess:  7.0
Average Load:  10.0


We obtain a rate similar to human performance but with a decently low learning load.

---

